<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/015-Scaling-ANY-Raster-From-GEE-STAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scaling and Offseting ANY RASTER DATASET From the GEE STAC

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/tutorials](https://github.com/davemlz/eemont/tree/master/tutorials)

## Let's start!

If required, please uncomment:

In [ ]:
#!pip install eemont
#!pip install geemap

Import the required packges.

In [ ]:
import ee, eemont, geemap
import geemap.colormaps as cm

Authenticate and Initialize Earth Engine and geemap.

In [ ]:
Map = geemap.Map()

Let's define a point of interest:

In [ ]:
poi = ee.Geometry.PointFromQuery("Argel, Algeria",user_agent = "eemont-tutorial-015")

Let's try 5 different datasets:

In [ ]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(poi)
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(poi)
MOD11A2 = ee.ImageCollection("MODIS/006/MOD11A2").filterBounds(poi)
WAPOR_ET = ee.ImageCollection("FAO/WAPOR/2/L1_AETI_D").filterBounds(poi)
NOAA_OCEAN_SURF_TEMP = ee.ImageCollection("NOAA/CDR/OISST/V2_1").filterBounds(poi)

## Check the SCALE parameters

In order to check the SCALE parameters of any raster dataset (ee.Image or ee.ImageCollection), use the `getScaleParams`:

In [ ]:
S2.getScaleParams()

The `getScaleParams` will retrieve a dictionay where each key is each of the bands of the datasets and the value is the scale factor for that band.

The `getScaleParams` method also works for ee.Image classes:

In [ ]:
S2.first().getScaleParams()

If a band doesn't have a scale factor, this will be retrieved as 1.

Now, let's check the scale parameters for the rest of the datasets:

In [ ]:
L8.getScaleParams()

In [ ]:
MOD11A2.getScaleParams()

In [ ]:
WAPOR_ET.getScaleParams()

In [ ]:
NOAA_OCEAN_SURF_TEMP.getScaleParams()

## Check the OFFSET parameters

In order to check the OFFSET parameters of any raster dataset (ee.Image or ee.ImageCollection), use the `getOffsetParams`:

In [ ]:
S2.getOffsetParams()

The `getOffsetParams` will retrieve a dictionay where each key is each of the bands of the datasets and the value is the offset factor for that band.

The `getOffsetParams` method also works for ee.Image classes:

In [ ]:
S2.first().getOffsetParams()

If a band doesn't have an offset factor, this will be retrieved as 0.

Now, let's check the offset parameters for the rest of the datasets:

In [ ]:
L8.getOffsetParams()

In [ ]:
MOD11A2.getOffsetParams()

Most datasets have no offset factors, however, in some cases (e.g. MOD11A2), these can be found.

In [ ]:
WAPOR_ET.getOffsetParams()

In [ ]:
NOAA_OCEAN_SURF_TEMP.getOffsetParams()

## Automatic Scaling and Offseting

The automatic scaling and offseting can be achieved by running the `scaleAndOffset` method for ee.Image and ee.ImageCollection classes (This method will eventually replace the `scale` method):

In [ ]:
S2 = S2.scaleAndOffset()
L8 = L8.scaleAndOffset()
MOD11A2 = MOD11A2.scaleAndOffset()
WAPOR_ET = WAPOR_ET.scaleAndOffset()
NOAA_OCEAN_SURF_TEMP = NOAA_OCEAN_SURF_TEMP.scaleAndOffset()

Visualization parameters:

In [ ]:
reflectance_RGB_vis = {
    "min": 0,
    "max": 0.3,
    "bands": ["B4","B3","B2"]
}

Kelvin_LST_vis = {
    "min": 280,
    "max": 320,
    "bands": "LST_Day_1km",
    "palette": cm.palettes.inferno
}

mm_ET_vis = {
    "min": 0,
    "max": 5,
    "bands": "L1_AETI_D",
    "palette": cm.palettes.Blues
}

Celsius_OST_vis = {
    "min": -1.8,
    "max": 30,
    "bands": "sst",
    "palette": cm.palettes.inferno
}

Let's visualize everything with `geemap`:

In [ ]:
Map = geemap.Map()
Map.addLayer(S2.first(),reflectance_RGB_vis,"S2 RGB (SR)")
Map.addLayer(L8.first(),reflectance_RGB_vis,"L8 RGB (SR)")
Map.addLayer(MOD11A2.first(),Kelvin_LST_vis,"MOD11A2 LST Day (Kelvin)")
Map.addLayer(WAPOR_ET.first(),mm_ET_vis,"WAPOR ET (mm)")
Map.addLayer(NOAA_OCEAN_SURF_TEMP.first(),Celsius_OST_vis,"NOAA OST (Celsius)")
Map.centerObject(poi,6)
Map